In [1]:
import cv2 as cv
import numpy as np
   
video = cv.VideoCapture('trafico.mp4')
bgimage = cv.bgsegm.createBackgroundSubtractorMOG()
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))
carcounter = 0
offset = 6
while True:
    _, frame = video.read()
    frame = cv.GaussianBlur(frame, (5, 5), 0)
    #frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    roi = frame[468:1080, 923:1920]
    roiframe = np.copy(roi)
	# Aplicamos suavizado para eliminar ruido
    
    fgMask = bgimage.apply(roiframe)
    fgMask = cv.morphologyEx(fgMask, cv.MORPH_OPEN, kernel)
    fgMask = cv.dilate(fgMask, None, iterations = 4)
    contours, _ = cv.findContours(fgMask, cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)
    img = cv.cvtColor(fgMask, cv.COLOR_GRAY2BGR)
    
    meta = cv.line(roiframe, (65, 323), (842, 323), (255, 0, 0), 1)
    for cnt in contours:
    # get the bounding rect
        area = cv.contourArea(cnt)
        if area > 1900:
            x, y, w, h = cv.boundingRect(cnt)
            cv.rectangle(roiframe, (x, y), (x+w, y+h), (0, 255, 0), 3)
            if (323  - offset)< y < (323 + offset):
                carcounter = carcounter+1
                meta = cv.line(roiframe, (65, 323), (842, 323), (0, 0, 255), 1)
                cv.rectangle(roiframe, (x, y), (x+w, y+h), (255, 0, 255), 3)

    cv.putText(frame, "coches: "+ str(carcounter), (1200, 250), cv.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 3)

    frame[468:1080, 923:1920] = roiframe
    fgMask = cv.resize(fgMask, (320, 180))
    frame = cv.resize(frame, (1600, 900))
    cv.imshow('frame', frame)
    cv.imshow('mask', fgMask) #coches en movimiento detectados
    cv.imshow('roi', roiframe)
    k = cv.waitKey(3) & 0xFF
    if k == 27:
        break
 
cv.destroyAllWindows()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\smooth.dispatch.cpp:617: error: (-215:Assertion failed) !_src.empty() in function 'cv::GaussianBlur'


Imagen del fondo:

In [9]:
import cv2
import numpy as np

file_path = 'trafico.mp4'
cap = cv2.VideoCapture(file_path)
first_iter = True
result1 = None
while True:
    ret, frame = cap.read()
    if frame is None:
        break
    if first_iter:
       avg = np.float32(frame)
       first_iter = False
    cv2.accumulateWeighted(frame, avg, 0.005)
    result1 = cv2.convertScaleAbs(avg)
    cv2.imwrite('traficobackground.jpg', result1)

cv2.imshow("background", result1)
cv2.waitKey(0)

-1